In [4]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import input_file_name, col, split, lower, regexp_replace, explode, count
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number

# Iniciar la sesión de Spark
spark = SparkSession.builder.master("local[*]").appName("LibrosSpark").getOrCreate()

# Leer todos los archivos de texto en un DataFrame
libros_df = spark.read.text("/content/libros/*")

# Añadir una columna con el nombre del archivo de origen
libros_df = libros_df.withColumn("libro", input_file_name())

# Convertir texto a minúsculas, quitar símbolos, y separar en palabras
libros_df = libros_df.select("libro",
                             split(lower(regexp_replace(col("value"), r'[^a-zA-Záéíóúñü\s]', '')), r'\s+')
                             .alias("palabras"))

# Explode para tener una palabra por fila y filtrar palabras vacías
libros_df = libros_df.select("libro", explode(col("palabras")).alias("palabra")).filter(col("palabra") != "")

# Contar las ocurrencias de cada palabra por libro
conteo_df = libros_df.groupBy("libro", "palabra").count()

# Definir la ventana de ordenamiento por conteo y libro
window = Window.partitionBy("libro").orderBy(col("count").desc())

# Agregar la columna de ranking y filtrar para obtener las 5 palabras más frecuentes por libro
top_5_palabras_df = conteo_df.withColumn("rank", row_number().over(window)).filter(col("rank") <= 5)

# Mostrar el resultado
top_5_palabras_df.show(truncate=False)

# Detener la sesión de Spark
spark.stop()

+----------------------------------------------+-------+-----+----+
|libro                                         |palabra|count|rank|
+----------------------------------------------+-------+-----+----+
|file:///content/libros/don_quijote.txt        |que    |20767|1   |
|file:///content/libros/don_quijote.txt        |de     |18411|2   |
|file:///content/libros/don_quijote.txt        |y      |18271|3   |
|file:///content/libros/don_quijote.txt        |la     |10492|4   |
|file:///content/libros/don_quijote.txt        |a      |9933 |5   |
|file:///content/libros/la_celestina.txt       |que    |4293 |1   |
|file:///content/libros/la_celestina.txt       |de     |4187 |2   |
|file:///content/libros/la_celestina.txt       |e      |2297 |3   |
|file:///content/libros/la_celestina.txt       |la     |2253 |4   |
|file:///content/libros/la_celestina.txt       |a      |1922 |5   |
|file:///content/libros/lazarillo_de_tormes.txt|y      |1154 |1   |
|file:///content/libros/lazarillo_de_tormes.txt|